In [5]:
# 先获取原始邮件文件
# 读取并存储原始邮件代码
import os
import chardet
from service import EmailReceiver, EmailSender
from config import config

# 配置保存邮件的目录
EMAIL_DIR = os.path.join("data", "emails")

def decode_email_content(payload: bytes) -> str:
    """
    尝试使用动态检测和指定编码解码邮件正文。
    """
    try:
        # 尝试使用 UTF-8 解码
        return payload.decode("utf-8")
    except UnicodeDecodeError:
        # 动态检测编码
        detected = chardet.detect(payload)
        encoding = detected.get("encoding") or "utf-8"
        try:
            return payload.decode(encoding)
        except Exception as e:
            print(f"Failed to decode with detected encoding {encoding}: {e}")
            return "[UNABLE TO DECODE CONTENT]"

def save_email(email_message, email_id, save_dir):
    """
    保存邮件内容到本地文件，动态处理编码问题。
    """
    try:
        # 检查邮件是否已经保存
        email_id_str = email_id.decode()  # IMAP 的邮件ID是字节，需要解码
        filename = f"{email_id_str}.txt"
        filepath = os.path.join(save_dir, filename)

        if os.path.exists(filepath):
            print(f"Email ID {email_id_str} already saved, skipping.")
            return

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # 确保文件名不包含特殊字符
        subject = email_message.get("subject", "No Subject").replace("/", "_").replace("\\", "_")

        with open(filepath, "w", encoding="utf-8") as f:
            # 保存邮件头部
            f.write(f"Subject: {subject}\n")
            f.write(f"From: {email_message.get('from')}\n")
            f.write(f"To: {email_message.get('to')}\n")
            f.write(f"Date: {email_message.get('date')}\n\n")

            # 保存邮件正文
            if email_message.is_multipart():
                # 遍历所有邮件部分，查找 text/plain 部分
                for part in email_message.walk():
                    if part.get_content_type() == "text/plain":
                        content = part.get_payload(decode=True)
                        if content:
                            f.write(decode_email_content(content))
                            break
            else:
                # 非多部分邮件，直接处理邮件正文
                content = email_message.get_payload(decode=True)
                if content:
                    f.write(decode_email_content(content))

        print(f"Email ID {email_id_str} saved to {filepath}")
    except Exception as e:
        print(f"Error saving email ID {email_id.decode()}: {e}")


email_receiver = EmailReceiver(config.IMAP_SERVER, config.IMAP_PORT, config.EMAIL_USERNAME, config.EMAIL_USERNAME)


try:
    email_receiver.connect()
    email_ids = email_receiver.fetch_all_emails()
    print(f"Found {len(email_ids)} emails.")

    for email_id in email_ids:
        email_message = email_receiver.fetch_email_by_id(email_id)
        if email_message:
            save_email(email_message, email_id, EMAIL_DIR)
finally:
    email_receiver.disconnect()


Exception: Error connecting to IMAP server: b'LOGIN Login error or password error'